In [1]:
#imports
import numpy as np

In [25]:
#load small input
input = open("./../../data/day_10/small_input.txt", "r").readlines()

In [12]:
#load small input 2
input = open("./../../data/day_10/small2_input.txt", "r").readlines()

In [54]:
#load small input 3
input = open("./../../data/day_10/small3_input.txt", "r").readlines()

In [62]:
#load small input 4
input = open("./../../data/day_10/small4_input.txt", "r").readlines()

In [70]:
#load input
input = open("./../../data/day_10/input.txt", "r").readlines()

In [71]:
#process input
matrix = np.zeros((len(input), len(input[0].strip())), dtype=np.dtypes.StrDType)

for i,line in enumerate(input):
    line = line.strip()
    for j,char in enumerate(line):
        matrix[i,j] = char
        if char == "S":
            start = (i,j)



In [72]:
#helper functions
def get_connections(c):
    """ Return tuple of connection booleans (left, up, right, down) for a given character c
    """
    match c:
        case '|': return (False, True, False, True)
        case '-': return (True, False, True, False)
        case 'L': return (False, True, True, False)
        case 'J': return (True, True, False, False)
        case '7': return (True, False, False, True)
        case 'F': return (False, False, True, True)
        case _: return (False, False, False, False)

LEFT = 0
UP = 1
RIGHT = 2
DOWN = 3

def get_reverse_direction(direction):
    if direction == LEFT:
        return RIGHT
    if direction == UP:
        return DOWN
    if direction == RIGHT:
        return LEFT
    if direction == DOWN:
        return UP

def get_next_pos(pos, direction):
    (x,y) = pos
    if direction == LEFT:
        return (x,y-1)
    if direction == UP:
        return (x-1,y)
    if direction == RIGHT:
        return (x,y+1)
    if direction == DOWN:
        return (x+1,y)


In [73]:
# part 1
# find loop
loop = []
#find symbol of start position (assume that there are only two pipes connecting to the start position, otherwise this will be more difficult)
(x,y) = start
if get_connections(matrix[x-1,y])[DOWN] and get_connections(matrix[x+1,y])[UP]:
    #start is connected to up and down
    matrix[x,y] = '|'
elif get_connections(matrix[x,y-1])[RIGHT] and get_connections(matrix[x,y+1])[LEFT]:
    #start is connected to left and right
    matrix[x,y] = '-'
elif get_connections(matrix[x-1,y])[DOWN] and get_connections(matrix[x,y-1])[RIGHT]:
    #start is connected to up and left
    matrix[x,y] = 'J'
elif get_connections(matrix[x-1,y])[DOWN] and get_connections(matrix[x,y+1])[LEFT]:
    #start is connected to up and right
    matrix[x,y] = 'L'
elif get_connections(matrix [x+1,y])[UP] and get_connections(matrix[x,y-1])[RIGHT]:
    #start is connected to down and left
    matrix[x,y] = '7'
elif get_connections(matrix [x+1,y])[UP] and get_connections(matrix[x,y+1])[LEFT]:
    #start is connected to down and right
    matrix[x,y] = 'F'

#traverse loop
loop.append((matrix[x,y], (x,y)))
connections = get_connections(matrix[x,y])
direction = -1
for i,v in enumerate(connections):
    if v:
        direction = i
        break
(x,y) = get_next_pos((x,y), direction)
while not (x,y) == start:
    loop.append((matrix[x,y], (x,y)))
    connections = get_connections(matrix[x,y])
    in_dir = get_reverse_direction(direction)
    for i,v in enumerate(connections):
        if v and not i == in_dir:
            direction = i
            break
    (x,y) = get_next_pos((x,y), direction)
    
steps_to_furthest = len(loop)/2
print(f"Part 1: {steps_to_furthest} steps to furthest point in the loop.")

Part 1: 6697.0 steps to furthest point in the loop.


In [ ]:
"""
ideas for part 2:
-> first, expand the loop -> add 0s along the left and the right of the loop, which allow calculating the squeezing between pipes
-> then, add a perimeter of virtual tiles that are certainly not in the loop around the border of the matrix
-> then, flood the matrix from the border. If a tile is not part of the loop and has a connection to some other tile that is free (already flooded), then mark it as free as well
"""

In [74]:
#expand the loop with zeros along the direction of the pipes
def expand_tile(c):
    expanded = np.zeros((3,3), dtype=np.dtypes.StrDType)
    expanded[1,1] = c
    connections = get_connections(c)
    if connections[LEFT]:
        expanded[1,0] = '-'
    if connections[RIGHT]:
        expanded[1,2] = '-'
    if connections[UP]:
        expanded[0,1] = '|'
    if connections[DOWN]:
        expanded[2,1] = '|'
    return expanded

def is_real_position(pos):
    (x,y) = pos
    return (x>=2) and (y>=2) and (x-2)%3==0 and (y-2)%3==0

def position_in_loop(pos):
    return pos in [p for _,p in loop]

In [75]:
expanded = np.zeros((matrix.shape[0]*3, matrix.shape[1]*3), dtype=np.dtypes.StrDType)
for x in range(matrix.shape[0]):
    for y in range(matrix.shape[1]):
        e = expand_tile(matrix[x,y]) if position_in_loop((x,y)) else np.zeros((3,3), dtype=np.dtypes.StrDType)

        expanded[x*3:(x*3)+3,y*3:(y*3)+3] = e

In [76]:
#add perimeter
expanded_p = np.zeros((expanded.shape[0]+2, expanded.shape[1]+2), dtype=np.dtypes.StrDType)
expanded_p[1:-1,1:-1] = expanded

In [77]:
#perform bfs
visited = np.zeros(expanded_p.shape, dtype=bool)
visited[0,0] = True

def get_neighbours(pos,matrix_shape):
    x,y = pos
    xm,ym = matrix_shape
    neighbours = []
    if x > 0:
        neighbours.append((x-1,y))
    if y > 0:
        neighbours.append((x,y-1))
    if x < xm-1:
        neighbours.append((x+1,y))
    if y < ym-1:
        neighbours.append((x,y+1))
    return neighbours

queue = [(0,0)]
while len(queue) > 0:
    x,y = queue.pop(0)
    for nx,ny in get_neighbours((x,y), expanded_p.shape):
        if (not visited[nx,ny]) and (expanded_p[nx,ny] == 0): #for some reason accessing expanded_p here returns an int instead of some string type, therefore the check works with == 0 instead of == '0'
            visited[nx,ny] = True
            queue.append((nx,ny))    
#count tiles
enclosed = 0
for i in range(visited.shape[0]):
    for j in range(visited.shape[1]):
        if not visited[i,j] and is_real_position((i,j)):
            enclosed += 1
enclosed = enclosed - len(loop)

print(f"Part 2: there are {enclosed} enclosed tiles.")

Part 2: there are 423 enclosed tiles.
